# 5 Despliegue con microservicios

## 5.1 Configuración de InfluxDB, Kapacitor, UDF y Grafana

1. Abrir dashboard [Grafana - analisis-de-fraude-en-operaciones-con-tarjetas-de-credito](http://localhost:3000/d/OSt1HwhMz3/analisis-de-fraude-en-operaciones-con-tarjetas-de-credito?orgId=1&refresh=1s). (En caso de que no esté importarlo del directorio 'dashboards' adentro de compose/grafana/dashboards).

Por problemas en la secuencia de inicio de servicios, a veces Kapacitor y el agente de Kapacitor para las User Defined Functions (UDF)  pueden no haberse iniciado por errores de conexión o de acceso a recursos. Para solucionarlo, basta con reiniciarlos en este orden:

2. Reiniciar UDF de kapacitor

~~~bash
cd compose
./restart-kapacitor-udf.sh
~~~

Debe verse: 

~~~bash
2020-12-29 02:02:12,257 INFO:root: Started server. Listening at: /tmp/comm/udf_ml_model.sock
~~~

3. Reiniciar kapacitor.

~~~bash
cd compose
./restart-kapacitor.sh
~~~

4. Conectarse a Cronograph y verificar que la conexión con InfluxDB y Kapacitor está establecida.
5. Verificar que el TICKScript FraudDetection está habilitado.

## 5.2 Inyección de datos (mock)

In [6]:
DATASET_PATH = '/data/credit_fraud/'

In [7]:
import pandas as pd
import numpy as np

In [8]:
test_df = pd.read_csv(DATASET_PATH+"creditcard_test.csv")

In [9]:
non_feature_cols = ['Unnamed: 0','time','row_id','class']
feature_cols = [x for x in test_df.columns if x not in non_feature_cols]
feature_cols

['v1',
 'v2',
 'v3',
 'v4',
 'v5',
 'v6',
 'v7',
 'v8',
 'v9',
 'v10',
 'v11',
 'v12',
 'v13',
 'v14',
 'v15',
 'v16',
 'v17',
 'v18',
 'v19',
 'v20',
 'v21',
 'v22',
 'v23',
 'v24',
 'v25',
 'v26',
 'v27',
 'v28',
 'amount']

Mock para inyectar valores de prueba en InfluxDB

In [10]:
import threading
import time
import math
import datetime
from influxdb import InfluxDBClient

class InfluxDBPublisher(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.keep_running = True
        self.client = InfluxDBClient(host='influxdb', port=8086, username='root', password='root',
                                     database="my_application")
    def run(self):
        while self.keep_running:
            time.sleep(0.5)
            json_body = []
            ts = datetime.datetime.utcnow()
            row = test_df.sample(1)
            fields = {}
            for f in feature_cols:
                fields[f] = row[f].values[0]             
            json_body.append(
                {
                    "measurement": f"TMV_CREDIT_OPERATION",
                    "time": ts,
                    "fields": fields
                }
            )
            self.client.write_points(json_body)

    def stop(self):
        self.keep_running = False
        self.join()


pub = InfluxDBPublisher()
pub.start()
input("Press any key to stop")
pub.stop()

Press any key to stop
